<a href="https://colab.research.google.com/github/FilledEther20/Feature_Selection_Hybrid_MPA-CFHO/blob/main/Feature_Selection_Intern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Docs Initialized



In [ ]:
!pip install mealpy
import numpy as np
import mealpy
from mealpy import FloatVar, MPA, CRO, PSO
import matplotlib.pyplot as plt
import random
# print(mealpy.__version__)
# print(np.__version__)
import logging
# Set logging level to WARNING or ERROR to suppress INFO messages
logging.getLogger('mealpy').setLevel(logging.WARNING)

#Chaotic Maps Defined

In [ ]:
def logistic_map(x, r=3.99):
      return r * x * (1 - x)

def tent_map(x, mu=1.99):
      if x < 0.5:
          return mu * x
      else:
          return mu * (1 - x)

#Pure Fire Hawk Optimizer


In [ ]:
class FHO():
    """ Class for FHO
    ---------
    self.cost_function: function to optimize
    self.Pop : np.array of Generated Solution Candidates of shape (pop_size,n_dims)
    self.n_dims: dimension of the problem
    self.pop_size : Number of solution candidates (int)
    self.min_bounds : numpy array of dimension (n_dims,) containing the MIN value for each variable
    self.max_bounds : numpy array of dimension (n_dims,) containing the MAX value for each variable
    self.max_generations: Maximum number of generated candidates (int)
    self.best_costs: python list of best costs (list)
    self.costs_iter : python list of cost calculated in each MAIN iteration (list)
    self.minimal_p : minimal point as a numpy array of dimensions (n_dims,)
    self.path :  A list containing the trajectory of the best hawks
    """

    def __init__(self, min_bounds , max_bounds, pop_size, cost_function , max_generations=200):
        self.max_generations = max_generations
        self.min_bounds      = min_bounds
        self.max_bounds      = max_bounds
        self.cost_function   = cost_function
        self.pop_size        = pop_size
        self.n_dims          = len(min_bounds) #Number of decision variables
        self.Pop             = np.random.uniform(min_bounds,max_bounds,(pop_size,self.n_dims)) #Initial Solution candidates (pop_size,n_dims)
        self.best_costs      = []
        self.costs_iter      = []
        self.minimal_p       = None
        self.path            = []

    def territories(self,Fire_Hawks,Preys):
        '''
        Inputs:
        self
        Fire_Hawks : np.array of fire hawks of dim (num_Hawks,self.n_dims)
        Preys : np.array of preys od dim (self.pop_size-num_Hawks,self.n_dims)

        Output:
        territories : territory of each hawk in a dictionary of nested numpy arrays
        '''
        #Computing territories using the euclidien distance
        preys_left=Preys.copy()
        territories={i:np.array([]) for i in range(len(Fire_Hawks))}
        for i in range(len(Fire_Hawks)):
            #distance with respect to Fire hawk i
            D=np.linalg.norm(Fire_Hawks[i]-preys_left,axis=1)

            #Get territory of fire Hawk i
            sorted_preys_idx=np.argsort(D)
            alpha=np.random.randint(1,len(preys_left)-1) if len(preys_left)-1>1 else 1
            my_preys=sorted_preys_idx[:alpha]
            territories[i]=preys_left[my_preys]
            preys_left=preys_left[sorted_preys_idx[alpha:]]
            if len(preys_left)==0:
                break
        if len(preys_left)>0:
            territories[len(Fire_Hawks)-1]=np.array(list(territories[len(Fire_Hawks)-1])+list(preys_left))
        return territories


    def minimize_FHO(self):
        '''
        Input:
        self

        Output:
        GB : global best solution (min f(x))
        self.minimal_p minimal point as a numpy array of dimensions (n_dims,)
        '''
        ## Fire hawk algorithm to minimize the cost function
        n_dims=self.n_dims
        Pop=self.Pop
        pop_size=self.pop_size
        max_generations = self.max_generations
        min_bounds=self.min_bounds
        max_bounds=self.max_bounds
        cost_function = self.cost_function

        #Evaluate the cost function for all candidate vectors
        cost= np.array([cost_function(Pop[i]) for i in range(pop_size)])

        #Randomly set a number of Hawks between 1 and 20% of pop_size
        num_Hawks = np.random.randint(1,int(pop_size/5)+1) if 1<int(pop_size/5)+1 else 1

        #Ordering candidates
        Pop = Pop[np.argsort(cost)]
        cost.sort()
        SP=Pop.mean(axis=0)


        #Select fire hawks
        Fire_Hawks= Pop[:num_Hawks]

        #Select the Preys dim(pop_size-num_Hawks,n_dims)
        Preys = Pop[num_Hawks:]

        #get territories
        territories=self.territories(Fire_Hawks,Preys)

        #update best
        GB=cost[0]
        Best_Hawk=Pop[0]
        self.path.append(Best_Hawk)

        #Counter
        FEs=pop_size

        ## Main Loop
        while FEs < max_generations:
            Pop_Tot=[]
            cost=[]
            #Movement of Fire Hawk for all territories
            for i in territories:
                PR=territories[i].copy()
                FHl=Fire_Hawks[i].copy()
                SPl=PR.mean(axis=0) if len(territories[i]) > 0 else np.zeros(FHl.shape)
                a,b=np.random.uniform(0,1,size=2)
                FHnear  = Fire_Hawks[np.random.randint(num_Hawks)]
                FHl_new = FHl+(a*GB-b*FHnear)
                FHl_new = np.maximum(FHl_new,min_bounds)
                FHl_new = np.minimum(FHl_new,max_bounds)
                Pop_Tot.append(list(FHl_new))

                #Movement of the preys following Fire Hawks movement
                for q in range(len(PR)):
                    a,b=np.random.uniform(0,1,size=2)
                    PRq_new1=PR[q].copy()+((a*FHl-b*SPl))
                    PRq_new1= np.maximum(PRq_new1,min_bounds)
                    PRq_new1 = np.minimum(PRq_new1,max_bounds)
                    Pop_Tot.append(list(PRq_new1))

                    #Movement of the preys outside of territory
                    a,b      =np.random.uniform(0,1,size=2)
                    FHAlter  =Fire_Hawks[np.random.randint(num_Hawks)]
                    PRq_new2 =PR[q].copy()+((a*FHAlter-b*SP));
                    PRq_new2 = np.maximum(PRq_new2,min_bounds)
                    #The following line for PRq_new2 differs from original algorithm in matlab code (max instead of min):
                    # Effects observed through our testing:
                    # 1/ It converges faster and the costs of the subsequent iterations will tend to decrease (less chaotic behavior than with np.minimun)
                    # 2/ In higher dimensions, it converge to the right solution! (while with np.minimum it does not)
                    PRq_new2 = np.maximum(PRq_new2,max_bounds)
                    Pop_Tot.append(list(PRq_new2))

            #Get cost
            Pop_Tot=np.array(Pop_Tot)
            for i in range(len(Pop_Tot)):
                cost.append(cost_function(Pop_Tot[i]))
                FEs = FEs+1

            #Create a new population of Hawks and Preys
            order_idx=np.argsort(cost)
            cost.sort()
            Pop_Tot=np.array(Pop_Tot)[order_idx]
            num_Hawks = np.random.randint(1,int(pop_size/5)+1) if 1<int(pop_size/5)+1 else 1
            Best_Pop=Pop_Tot[0]
            SP=Pop_Tot.mean(axis=0)
            Fire_Hawks=Pop_Tot[:num_Hawks]
            Preys=Pop_Tot[num_Hawks:]

            #Get new territories
            territories=self.territories(Fire_Hawks,Preys)

            # Update Global Best cost (if relevant)
            if cost[0]<GB:
                Best_Position=Best_Pop
                GB=cost[0]
                self.best_costs.append(GB)
                self.minimal_p=Fire_Hawks[0]
                self.path.append(Best_Position)
            else:
                self.best_costs.append(GB)

            #Track the iteration calculated cost
            self.costs_iter.append(cost[0])

        #Return Global Best and argmin
        return (GB,self.minimal_p)
    def plot_costs(self):
        #Plot cost evolution
        vals=self.costs_iter
        n=len(vals)
        vals2=self.best_costs
        plt.figure()
        plt.title("Cost per iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Cost")
        plt.plot(np.arange(n),vals,label="Iteration's calculated Cost")
        plt.plot(np.arange(n),vals2, label="Global Best Cost")
        plt.xticks(np.arange(n))
        plt.legend()
        plt.show()

    def plot_log_costs(self):
        #Plot cost evolution
        vals=np.log(self.costs_iter)
        n=len(vals)
        vals2=self.best_costs
        plt.figure()
        plt.title("Log Cost per iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Cost")
        plt.plot(np.arange(n),vals,label="Log Cost")
        plt.xticks(np.arange(n))
        plt.legend()
        plt.show()

    def get_path(self):
      return np.array(self.path).T


#Chaotic Fire Hawk Optimizer

In [ ]:
class Chaotic_FHO:
    def __init__(self, min_bounds , max_bounds, pop_size, cost_function , max_generations=200):
        self.max_generations = max_generations
        self.min_bounds      = min_bounds
        self.max_bounds      = max_bounds
        self.cost_function   = cost_function
        self.pop_size        = pop_size
        self.n_dims          = len(min_bounds) #Number of decision variables
        self.Pop             = np.random.uniform(min_bounds,max_bounds,(pop_size,self.n_dims)) #Initial Solution candidates (pop_size,n_dims)
        self.best_costs      = []
        self.costs_iter      = []
        self.minimal_p       = None
        self.path            = []

    def territories(self,Fire_Hawks,Preys):
        '''
        Inputs:
        self
        Fire_Hawks : np.array of fire hawks of dim (num_Hawks,self.n_dims)
        Preys : np.array of preys od dim (self.pop_size-num_Hawks,self.n_dims)

        Output:
        territories : territory of each hawk in a dictionary of nested numpy arrays
        '''
        #Computing territories using the euclidien distance
        preys_left=Preys.copy()
        territories={i:np.array([]) for i in range(len(Fire_Hawks))}
        for i in range(len(Fire_Hawks)):
            #distance with respect to Fire hawk i
            D=np.linalg.norm(Fire_Hawks[i]-preys_left,axis=1)
            #Get territory of fire Hawk i
            sorted_preys_idx=np.argsort(D)
            alpha=np.random.randint(1,len(preys_left)-1) if len(preys_left)-1>1 else 1
            my_preys=sorted_preys_idx[:alpha]
            territories[i]=preys_left[my_preys]
            preys_left=preys_left[sorted_preys_idx[alpha:]]
            if len(preys_left)==0:
                break
        if len(preys_left)>0:
            territories[len(Fire_Hawks)-1]=np.array(list(territories[len(Fire_Hawks)-1])+list(preys_left))
        return territories


    def minimize_FHO(self):
        '''
        Input:
        self

        Output:
        GB : global best solution (min f(x))
        self.minimal_p minimal point as a numpy array of dimensions (n_dims,)
        '''
        ## Fire hawk algorithm to minimize the cost function
        n_dims=self.n_dims
        Pop=self.Pop
        pop_size=self.pop_size
        max_generations = self.max_generations
        min_bounds=self.min_bounds
        max_bounds=self.max_bounds
        cost_function = self.cost_function

        #Evaluate the cost function for all candidate vectors
        cost= np.array([cost_function(Pop[i]) for i in range(pop_size)])

        #Randomly set a number of Hawks between 1 and 20% of pop_size
        num_Hawks = np.random.randint(1,int(pop_size/5)+1) if 1<int(pop_size/5)+1 else 1

        #Ordering candidates
        Pop = Pop[np.argsort(cost)]
        cost.sort()
        SP=Pop.mean(axis=0)


        #Select fire hawks
        Fire_Hawks= Pop[:num_Hawks]

        #Select the Preys dim(pop_size-num_Hawks,n_dims)
        Preys = Pop[num_Hawks:]

        #get territories
        territories=self.territories(Fire_Hawks,Preys)

        #update best
        GB=cost[0]
        Best_Hawk=Pop[0]
        self.path.append(Best_Hawk)

        #Counter
        FEs=pop_size

        ## Main Loop
        while FEs < max_generations:
            Pop_Tot=[]
            cost=[]
            #Movement of Fire Hawk for all territories
            for i in territories:
                PR=territories[i].copy()
                FHl=Fire_Hawks[i].copy()
                SPl=PR.mean(axis=0) if len(territories[i]) > 0 else np.zeros(FHl.shape)
                a,b=np.random.uniform(0,1,size=2)
                FHnear = Fire_Hawks[np.random.randint(len(Fire_Hawks))]

                #Update Fire Hawk's position (FHl_new) using chaotic maps
                FHl_new = FHl + (a * GB - b * FHnear)
                FHl_new = np.maximum(FHl_new, min_bounds)
                FHl_new = np.minimum(FHl_new, max_bounds)
                Pop_Tot.append(list(FHl_new))

                # Apply chaotic map to perturb the new position
                chaos_factor = logistic_map(np.random.rand())
                FHl_new = FHl_new + chaos_factor * (np.random.rand(self.n_dims) - 0.5)
                FHl_new = np.maximum(FHl_new, min_bounds)
                FHl_new = np.minimum(FHl_new, max_bounds)
                Pop_Tot.append(list(FHl_new))
            for q in range(len(PR)):
                a,b=np.random.uniform(0,1,size=2)
                PRq_new1 = PR[q].copy() + ((a * FHl - b * SPl))
                PRq_new1 = np.maximum(PRq_new1, min_bounds)
                PRq_new1 = np.minimum(PRq_new1, max_bounds)

                # Apply chaotic map to perturb the prey's position
                chaos_factor = tent_map(np.random.rand())
                PRq_new1 = PRq_new1 + chaos_factor * (np.random.rand(self.n_dims) - 0.5)
                PRq_new1 = np.maximum(PRq_new1, min_bounds)
                PRq_new1 = np.minimum(PRq_new1, max_bounds)

                Pop_Tot.append(list(PRq_new1))

                # Movement of the preys outside of territory
                a,b = np.random.uniform(0,1,size=2)
                FHAlter = Fire_Hawks[np.random.randint(num_Hawks)]
                PRq_new2 = PR[q].copy() + ((a * FHAlter - b * SP))
                PRq_new2 = np.maximum(PRq_new2, min_bounds)
                PRq_new2 = np.minimum(PRq_new2, max_bounds)

                # Apply chaotic map to perturb the prey's alternative movement
                chaos_factor = logistic_map(np.random.rand())
                PRq_new2 = PRq_new2 + chaos_factor * (np.random.rand(self.n_dims) - 0.5)
                PRq_new2 = np.maximum(PRq_new2, min_bounds)
                PRq_new2 = np.minimum(PRq_new2, max_bounds)

                Pop_Tot.append(list(PRq_new2))

            #Get cost
            Pop_Tot=np.array(Pop_Tot)
            for i in range(len(Pop_Tot)):
                cost.append(cost_function(Pop_Tot[i]))
                FEs = FEs+1

            #Create a new population of Hawks and Preys
            order_idx=np.argsort(cost)
            cost.sort()
            Pop_Tot=np.array(Pop_Tot)[order_idx]
            num_Hawks = np.random.randint(1,int(pop_size/5)+1) if 1<int(pop_size/5)+1 else 1
            Best_Pop=Pop_Tot[0]
            SP=Pop_Tot.mean(axis=0)
            Fire_Hawks=Pop_Tot[:num_Hawks]
            Preys=Pop_Tot[num_Hawks:]

            #Get new territories
            territories=self.territories(Fire_Hawks,Preys)

            # Update Global Best cost (if relevant)
            if cost[0]<GB:
                Best_Position=Best_Pop
                GB=cost[0]
                self.best_costs.append(GB)
                self.minimal_p=Fire_Hawks[0]
                self.path.append(Best_Position)
            else:
                self.best_costs.append(GB)

            #Track the iteration calculated cost
            self.costs_iter.append(cost[0])

        #Return Global Best and argmin
        return (GB,self.minimal_p)
    def plot_costs(self):
        #Plot cost evolution
        vals=self.costs_iter
        n=len(vals)
        vals2=self.best_costs
        plt.figure()
        plt.title("Cost per iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Cost")
        plt.plot(np.arange(n),vals,label="Iteration's calculated Cost")
        plt.plot(np.arange(n),vals2, label="Global Best Cost")
        plt.xticks(np.arange(n))
        plt.legend()
        plt.show()

    def plot_log_costs(self):
        #Plot cost evolution
        vals=np.log(self.costs_iter)
        n=len(vals)
        vals2=self.best_costs
        plt.figure()
        plt.title("Log Cost per iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Cost")
        plt.plot(np.arange(n),vals,label="Log Cost")
        plt.xticks(np.arange(n))
        plt.legend()
        plt.show()

    def get_path(self):
      return np.array(self.path).T


# ChFHO-MPA Combined


In [ ]:
#The hybrid is created on the basis of the following attributes of individual algorithms:-
  #ChFHO : Due to chaos being introduced the exploration capabilities have been enhanced.
  #MPA : Due to the innate nature of the algorithm (optimal forging stratergy+(Levy Flight+Brownian Motion)) it is better equipped for exploitation phase.

#The working of the hybrid is divided into 2 major steps listed below:
  #First , Exploration is done through ChFHO for a certain number of iterations. This helps in identifying the most promising areas in the entire search space.
  #Second, Exploitation is done through MPA for the remaining number of iterations. This helps in exploiting the most promising areas.

def hybrid_chFHO_MPA(min_bounds, max_bounds, pop_size, cost_function, max_generations=200, chFHO_iter=100):
    # Initialize parameters
    n_dims = len(min_bounds)
    population = np.random.uniform(min_bounds, max_bounds, (pop_size, n_dims))

    # Initialize best solution and cost tracker
    best_cost = float("inf")
    best_solution = None
    best_costs = []

    # Run chFHO for the specified number of iterations
    for generation in range(chFHO_iter):
        # Update population using chFHO strategy
        population = chFHO_update(population, min_bounds, max_bounds, cost_function)

        # Evaluate costs and find the best solution
        costs = np.array([cost_function(ind) for ind in population])
        min_cost_index = np.argmin(costs)
        current_best_cost = costs[min_cost_index]
        current_best_solution = population[min_cost_index]

        # Update global best solution if current is better
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_solution = current_best_solution

        # Store best cost for tracking
        best_costs.append(best_cost)

        # Print progress (optional)
        if generation % 10 == 0:
            print(f"chFHO Generation {generation}: Best Cost = {best_cost}")

    # Run MPA for the remaining iterations
    for generation in range(chFHO_iter, max_generations):
        # Update population using MPA strategy
        population = MPA_update(population, min_bounds, max_bounds, cost_function)

        # Evaluate costs and find the best solution
        costs = np.array([cost_function(ind) for ind in population])
        min_cost_index = np.argmin(costs)
        current_best_cost = costs[min_cost_index]
        current_best_solution = population[min_cost_index]

        # Update global best solution if current is better
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_solution = current_best_solution

        # Store best cost for tracking
        best_costs.append(best_cost)

        # Print progress (optional)
        if generation % 10 == 0:
            print(f"MPA Generation {generation}: Best Cost = {best_cost}")

    # Return the best solution and cost evolution
    return best_solution, best_costs

# Placeholder functions for MPA and chFHO updates
def MPA_update(population, min_bounds, max_bounds, cost_function):
    # Implement the logic for updating the MPA population here
    return population  # Return the updated population

def chFHO_update(population, min_bounds, max_bounds, cost_function):
    # Implement the logic for updating the chFHO population here
    return population  # Return the updated population


# Benchmark Functions Defined

In [ ]:
# Sphere function objective
def sphere_function(solution):
    return np.sum(np.square(solution))

# # Rastirgin function objective
# def rastirgin_function(solution):
#   return 10*len(solution) + np.sum(np.square(solution)-10*np.cos(2*np.pi*solution))

# Griewank function objective
def griewank_function(solution):
  n=len(solution)
  sum_sq_term=np.sum(np.square(solution))
  product_term=np.prod(np.cos(solution/np.sqrt(np.arange(1,n+1))))
  return sum_sq_term/4000-product_term+1

# #Matya's function objective
# def matyas_function(x):
#     x1, x2 = x[:2]
#     return 0.26 * (x1**2 + x2**2) - 0.48 * x1 * x2

# Ackley function objective
def ackley_function(solution):
  n=len(solution)
  sum_sq_term=-0.2*np.sqrt(np.sum(solution**2)/n)
  cos_term=np.sum(np.cos(2*np.pi*solution))/n
  return -20*np.exp(sum_sq_term)-np.exp(cos_term)+20+np.exp(1)

# Rosenbrock function objective
def rosenbrock_function(solution):
      return np.sum(100 * (solution[1:] - solution[:-1]**2)**2 + (solution[:-1] - 1)**2)

# Levy function objective
def levy_function(solution):
    solution = np.asarray(solution)
    # Ensure that solution has at least two dimensions
    if len(solution) < 2:
        raise ValueError("Input vector must have at least two dimensions.")

    # Calculate w
    w = 1 + (solution - 1) / 4

    # Compute the Levy function
    term1 = np.sin(np.pi * w[0]) ** 2
    term2 = np.sum((w[:-1] - 1) ** 2 * (1 + 10 * np.sin(np.pi * w[:-1] + 1) ** 2))
    term3 = (w[-1] - 1) ** 2 * (1 + np.sin(2 * np.pi * w[-1]) ** 2)

    return term1 + term2 + term3





# Plotting Function Defined


In [ ]:
def plot_performance(fitness_values_dict, title, ylabel, filename=None):
    plt.figure(figsize=(10, 6))
    for label, values in fitness_values_dict.items():
        plt.plot(values, label=label)
    plt.yscale("log")
    plt.xlabel("Iterations")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    if filename:
        plt.savefig(filename)
    plt.show()


# Problem Dictionary Defined

In [ ]:

# Define problem dictionary for each function
problem_dicts = {
    "Sphere": {
        "obj_func": sphere_function,
        "bounds": FloatVar(lb=[-100] * 30, ub=[100] * 30, name="delta"),
        "minmax": "min",
        "save_population": True,
    },
    # "Matyas":{
    #     "obj_func": matyas_function,
    #     "bounds":  FloatVar(lb=[-10] * 30, ub=[10] * 30, name="delta"),
    #     "minmax": "min",
    #     "save_population": True,
    # },
    "Griewank": {
        "obj_func": griewank_function,
        "bounds": FloatVar(lb=[-600] * 30, ub=[600] * 30, name="delta"),
        "minmax": "min",
        "save_population": True,
    },
    "Ackley": {
        "obj_func": ackley_function,
        "bounds": FloatVar(lb=[-32.768] * 30, ub=[32.768] * 30, name="delta"),
        "minmax": "min",
        "save_population": True,
    },
    "Rosenbrock": {
        "obj_func": rosenbrock_function,
        "bounds": FloatVar(lb=[-5] * 30, ub=[5] * 30, name="delta"),
        "minmax": "min",
        "save_population": True,
    },
    "Levy": {
        "obj_func": levy_function,
        "bounds": FloatVar(lb=[-10] * 30, ub=[10] * 30, name="delta"),
        "minmax": "min",
        "save_population": True,
    },
}

# Define algorithms
def run_optimization(algorithm, problem_dict):
    optimizer = algorithm(
        epoch=250, pop_size=50, po=0.4, Fb=0.9, Fa=0.1, Fd=0.1, Pd=0.5,
        GCR=0.1, gamma_min=0.02, gamma_max=0.2, n_trials=5, restart_count=5
    )
    optimizer.solve(problem_dict)
    return optimizer.history.list_global_best_fit


# Testing & Plotting Graph


In [1]:
# Define the plotting for each function
for func_name, problem_dict in problem_dicts.items():
    if func_name=="Levy" or func_name=="Sphere" or func_name=="Griewank" or func_name=="Ackley":
      continue
    # CRO Optimization
    fitness_values_CRO = run_optimization(CRO.OCRO, problem_dict)

    # MPA Optimization
    fitness_values_MPA = run_optimization(MPA.OriginalMPA, problem_dict)

    # PSO Optimization
    fitness_values_PSO = run_optimization(PSO.OriginalPSO, problem_dict)



    # Chaotic_FHO Optimization
    minD = problem_dict['bounds'].lb
    maxD = problem_dict['bounds'].ub
    opti = Chaotic_FHO(minD, maxD, pop_size=50, cost_function=problem_dict['obj_func'], max_generations=6000)
    opti.minimize_FHO()
    fitness_values_CFHO = opti.costs_iter



    # Hybrid Algorithm Optimization
    fitness_values_hybrid = hybrid_chFHO_MPA(
        min_bounds=problem_dict['bounds'].lb,
        max_bounds=problem_dict['bounds'].ub,
        pop_size=50,
        cost_function=problem_dict['obj_func'],
        max_generations=6000,
        chFHO_iter=100  # Number of iterations for chFHO
    )

    # FHO Optimization
    opti_FHO=FHO(minD,maxD,50,problem_dict['obj_func'],2000)
    opti_FHO.minimize_FHO()
    fitness_values_FHO = opti_FHO.costs_iter

    # Plot results
    plot_performance({
        "CRO": fitness_values_CRO,
        "CFHO": fitness_values_CFHO,
        "MPA": fitness_values_MPA,
        "PSO": fitness_values_PSO,
        "FHO": fitness_values_FHO,
        "Hybrid": fitness_values_hybrid
    }, f"Logarithmic Fitness vs Iteration ({func_name} Function)", "Fitness (Cost)", f"{func_name.lower()}_performance.png")

NameError: name 'problem_dicts' is not defined